In [1]:
cd /content/drive/My Drive/Week7 Asssignment/

/content/drive/My Drive/Week7 Asssignment


# 1. Load Data

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
sample_submission = pd.read_csv("sample_submission.csv")
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")

In [4]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
test.head()

,Unnamed: 0,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,57808,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4960,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,35755,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,15543,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,48968,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
print(train.shape)
print(test.shape)

(42000, 785)
(18000, 785)


In [0]:
X_train = train.iloc[:,1:].values / 255
X_test = test.iloc[:,1:].values / 255
y_train = train.iloc[:,0].values

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

In [9]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(33600, 784) (33600,)
(8400, 784) (8400,)


# 2. Dataset, DataLoader

In [0]:
from torch.utils.data import  Dataset, DataLoader
from torchvision.transforms import Compose, RandomRotation

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

In [0]:
from torch.utils.data import Dataset, DataLoader

class TrainDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y

class TestDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        return X

In [0]:
traindataloader = DataLoader(TrainDataset(X_train, y_train), batch_size=1000, shuffle=True, num_workers=4)
valdataloader  = DataLoader(TrainDataset(X_val, y_val), batch_size=1000, shuffle=True, num_workers=4)
testdataloader = DataLoader(TestDataset(X_test, y=None), batch_size=4, shuffle=False, num_workers=4)

# 3. Model

## optimizer : Adam

In [0]:
class Model1(nn.Module):
  def __init__(self):
    super(Model1, self).__init__()
    self.fc1 = nn.Linear(784, 1024)
    self.fc2 = nn.Linear(1024, 512)
    self.dropout1 = nn.Dropout(p=0.2)
    self.fc3 = nn.Linear(512, 256)
    self.fc4 = nn.Linear(256, 128)
    self.dropout2 = nn.Dropout(p=0.2)
    self.fc5 = nn.Linear(128, 64)
    self.fc6 = nn.Linear(64, 10) 

  def forward(self, X):
    X = self.fc1(X)
    X = F.selu(X)
    X = self.fc2(X)
    X = F.selu(self.dropout1(X))
    X = self.fc3(X)
    X = F.selu(X)
    X = self.fc4(X)
    X = F.selu(self.dropout2(X))
    X = self.fc5(X)
    X = F.selu(X)
    X = self.fc6(X)
    return X

In [0]:
model1 = Model1()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters())

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [17]:
model1.to(device)

Model1(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc5): Linear(in_features=128, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=10, bias=True)
)

In [19]:
epochs = 200

for epoch in range(epochs):

  train_predictions = torch.tensor(0)
  train_y = torch.tensor(0)
  train_loss = 0;

  for i, data in enumerate(traindataloader):
    X, y = data
    X = torch.tensor(X, device=device).float()
    y = torch.tensor(y, device=device)

    optimizer.zero_grad() # gradient initialize
    output = model1(X)
    loss = criterion(output, y) # loss 
    train_loss += loss.item()
    loss.backward() # backpropagation
    optimizer.step() # parameter update

    if(i==0):
      train_predictions = output.argmax(dim=1)
      train_y = y
    else:
      train_predictions = torch.cat((train_predictions, output.argmax(dim=1)), dim=0)
      train_y = torch.cat((train_y, y), dim=0)
  train_accuracy = (train_predictions == train_y).sum().item() / len(train_y)

  if(epoch % 5 == 0):
    with torch.no_grad():
      X_val = torch.tensor(X_val, device=device).float()
      y_val = torch.tensor(y_val, device=device)
      val_predictions = model1(X_val)
      val_loss = criterion(val_predictions, y_val).item()
      val_predictions = val_predictions.argmax(dim=1)
      val_accuracy = (val_predictions == y_val).sum().item() / len(y_val)
      
    print("epoch : {} \t train_loss:{:.6f}, val_loss:{:.6f}, \t train_acc:{:.6f}, val_acc:{:.6f}".format(
        epoch, train_loss, val_loss, train_accuracy, val_accuracy))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


epoch : 0 	 train_loss:4.250580, val_loss:0.123276, 	 train_acc:0.961726, val_acc:0.962500


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


epoch : 5 	 train_loss:1.900658, val_loss:0.092448, 	 train_acc:0.981220, val_acc:0.974405
epoch : 10 	 train_loss:1.088549, val_loss:0.090565, 	 train_acc:0.989702, val_acc:0.976190
epoch : 15 	 train_loss:0.987872, val_loss:0.100895, 	 train_acc:0.990119, val_acc:0.974643
epoch : 20 	 train_loss:0.624434, val_loss:0.090058, 	 train_acc:0.994077, val_acc:0.978333
epoch : 25 	 train_loss:0.490264, val_loss:0.086626, 	 train_acc:0.995298, val_acc:0.980357
epoch : 30 	 train_loss:0.380562, val_loss:0.094575, 	 train_acc:0.995923, val_acc:0.980952
epoch : 35 	 train_loss:0.333105, val_loss:0.086423, 	 train_acc:0.996518, val_acc:0.980952
epoch : 40 	 train_loss:0.184413, val_loss:0.099709, 	 train_acc:0.997887, val_acc:0.982143
epoch : 45 	 train_loss:0.315356, val_loss:0.108730, 	 train_acc:0.996786, val_acc:0.978333
epoch : 50 	 train_loss:0.134120, val_loss:0.093593, 	 train_acc:0.998839, val_acc:0.982619
epoch : 55 	 train_loss:0.245174, val_loss:0.097646, 	 train_acc:0.997560, val_ac

## otimizer : SGD


In [0]:
model1 = Model1()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model1.parameters(), lr=0.001)

In [70]:
model1.to(device)

Model1(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc5): Linear(in_features=128, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=10, bias=True)
)

In [71]:
epochs = 400

for epoch in range(epochs):
  train_predictions = torch.tensor(0)
  train_y = torch.tensor(0)
  train_loss = 0;

  for i, data in enumerate(traindataloader):
    X, y = data
    X = torch.tensor(X, device=device).float()
    y = torch.tensor(y, device=device)

    optimizer.zero_grad()
    output = model1(X)
    loss = criterion(output, y)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()

    if(i==0):
      train_predictions = output.argmax(dim=1)
      train_y = y
    else:
      train_predictions = torch.cat((train_predictions, output.argmax(dim=1)), dim=0)
      train_y = torch.cat((train_y, y), dim=0)
  train_accuracy = (train_predictions == train_y).sum().item() / len(train_y)

  if(epoch % 5 == 0):
    with torch.no_grad():
      X_val = torch.tensor(X_val, device=device).float()
      y_val = torch.tensor(y_val, device=device)
      val_predictions = model1(X_val)
      val_loss = criterion(val_predictions, y_val).item()
      val_predictions = val_predictions.argmax(dim=1)
      val_accuracy = (val_predictions == y_val).sum().item() / len(y_val)
    print("epoch : {} \t train_loss:{:.6f}, val_loss:{:.6f}, \t train_acc:{:.6f}, val_acc:{:.6f}".format(
        epoch, train_loss, val_loss, train_accuracy, val_accuracy))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch : 0 	 train_loss:78.178939, val_loss:2.295423, 	 train_acc:0.101994, val_acc:0.107262


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


epoch : 5 	 train_loss:76.985254, val_loss:2.259828, 	 train_acc:0.163631, val_acc:0.179167
epoch : 10 	 train_loss:75.710166, val_loss:2.221468, 	 train_acc:0.273958, val_acc:0.293571
epoch : 15 	 train_loss:74.255976, val_loss:2.179092, 	 train_acc:0.426071, val_acc:0.439286
epoch : 20 	 train_loss:72.554727, val_loss:2.127688, 	 train_acc:0.565893, val_acc:0.576071
epoch : 25 	 train_loss:70.539762, val_loss:2.066034, 	 train_acc:0.659077, val_acc:0.663095
epoch : 30 	 train_loss:68.145862, val_loss:1.994785, 	 train_acc:0.713155, val_acc:0.710595
epoch : 35 	 train_loss:65.316813, val_loss:1.909877, 	 train_acc:0.744435, val_acc:0.745714
epoch : 40 	 train_loss:62.080990, val_loss:1.814647, 	 train_acc:0.757054, val_acc:0.760357
epoch : 45 	 train_loss:58.499819, val_loss:1.707398, 	 train_acc:0.769583, val_acc:0.765238
epoch : 50 	 train_loss:54.709166, val_loss:1.594481, 	 train_acc:0.779524, val_acc:0.778571
epoch : 55 	 train_loss:50.919062, val_loss:1.482390, 	 train_acc:0.789